In [ ]:
import imutils
import cv2
from PIL import *
from pytesseract import *
import time
from skimage.filter import threshold_adaptive
import numpy as np
import Image
import os
import Adafruit_CharLCD as LCD
import RPi.GPIO as GPIO
from datetime import datetime
import subprocess
import re

speaking = False

def speak(text):
    os.system("pico2wave --lang=en-US -w sample2.wav \"" + text + "\" && aplay sample2.wav")
    
def read(text):   
    p = subprocess.call("pico2wave --lang=en-US -w sample.wav \"" + text + "\"",shell = True)
    player = subprocess.Popen(["aplay", "sample.wav"])
        
def isr(channel):
    read("")
        
button = 2

GPIO.setup(button,GPIO.IN)

GPIO.add_event_detect(button,GPIO.FALLING,callback=isr,bouncetime=1000)

# Raspberry Pi pin configuration:
lcd_rs        = 21 
lcd_en        = 20
lcd_d4        = 16
lcd_d5        = 12
lcd_d6        = 7
lcd_d7        = 8


# Define LCD column and row size for 16x2 LCD
lcd_columns = 16
lcd_rows    = 2

lcd = LCD.Adafruit_CharLCD(lcd_rs, lcd_en, lcd_d4, lcd_d5, lcd_d6, lcd_d7,
                           lcd_columns, lcd_rows)

IMAGE_FILE='sample.jpg'

reset = True


while reset == True:

    try:

        main = True
        lcd.set_backlight(0)
        lcd.clear()
        lcd.message('\nImage Reader using Raspberry pi')
        speak("Image Reader using Raspberry pi")
        time.sleep(2)
        
        if main:    
            lcd.clear()
            lcd.message('Searching for\nCamera')
            speak("Searching for Camera.")
            time.sleep(2)

            camera=cv2.VideoCapture(0)

            ret = camera.set(3,640)
            ret = camera.set(4,480)
            
            if not camera.isOpened():
                main = False
                lcd.clear()
                lcd.message('Error:Camera not\nFound')
                time.sleep(2)
                lcd.clear()
                lcd.message('Connect Camera &\npress reset')
                speak("Camera not Found, Connect Camera & press reset.")
                while GPIO.input(button) == True:
                    None
            
            else:
                main = True
                lcd.clear()
                lcd.message('Found Camera')
                speak("camera founded.")
                time.sleep(2)
                camera.release()
                
        while main:
                                
            lcd.clear()
            lcd.message('Press switch...')
            speak("Place the paper, and press the switch")
            time.sleep(1)    
            
            while GPIO.input(button) == True:
                    None

            camera=cv2.VideoCapture(0)

            lcd.clear()
            lcd.message("Capturing Image.")
            speak("Capturing Image")
            time.sleep(2)
            
            ret, img = camera.read()

            camera.release()

            warped = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            warped = warped.astype("uint8") * 255

            time.sleep(1)
            lcd.clear()
            lcd.message("Image captured")
            speak("Image captured.")
            time.sleep(1)
            
            lcd.clear()
            lcd.message('Processing...')
            speak("Processing for the speech.")
            time.sleep(2)
            cv2.imwrite(IMAGE_FILE,warped)

            img = Image.open(IMAGE_FILE)

            words = image_to_string(img)
                                        
            if len(words) > 0:

                speak("Process complete, Press switch to \nstop playing.")
                words = words.replace('\n',' ')
                speech = re.sub('[^a-zA-Z0-9 \n\.\,]','',words)
                read(speech)
                time.sleep(2)
                lcd.clear()
                lcd.message('Press switch to \nstop playing')
                speak("speech completed.")
                
            else:
                
                lcd.clear()
                lcd.message('No text Found...')
                speak("No text Found.")
                time.sleep(3)
                        
                       
            if GPIO.input(button) == False:
                    break
                
    except Exception as e:
        lcd.clear()
        lcd.message('   !!!!Error!!!!   ')

        while GPIO.input(button):
            lcd.set_cursor(0,1)
            lcd.message('    Press reset     ')
            time.sleep(0.5)
            lcd.set_cursor(0,1)
            lcd.message('                    ')
            time.sleep(0.5)
